# Quaterly Alignment File

In [1]:
import pandas as pd
import polars as pl
import numpy as np

In [2]:
# Update Variables - 
QTR	= '2024Q4'
QTRP = '2024Q3'
UPDATE = '20240812'

FILE1= '2024_Q4_Final_Alignment_ZIP_Terr_formatted'
SHEET1='2024_Q4_Final_Alignment_ZIP_Ter'
FILE2='Hierarchy 2024Q4 93 Territories_formatted'
SHEET2='Hierarchy'

ZIP = f's3://vortex-staging-a65ced90/PYADM/quaterly/{QTR}/geography/'
#ZIP_P = f's3://vortex-staging-a65ced90/PYADM/quaterly/{QTRP}/geography/'

In [3]:
# Importing Raw FILE - 
ZIP_DF = pl.from_pandas(pd.read_excel(
    f's3://vortex-staging-a65ced90/PYADM/raw/{UPDATE}/quaterly_raw/{FILE1}.xlsx',
    sheet_name= SHEET1
))

HIERARCHY_DF = pl.from_pandas(pd.read_excel(
    f's3://vortex-staging-a65ced90/PYADM/raw/{UPDATE}/quaterly_raw/{FILE2}.xlsx',
    sheet_name= SHEET2
))

HIERARCHY - 

In [4]:
# Processing - 
HIERARCHY1 = (
    HIERARCHY_DF
    .filter(
        (~pl.col('Territory_ID').is_in(['1111-99999-99','1111-99999-11'])) & (pl.col('Territory_ID').is_not_null())
    )
    .rename({'Territory_ID':'Territory'})
)

# EXPORT - 
HIERARCHY1.to_pandas().to_parquet(f'{ZIP}hierarchy.parquet')

ZIP TO TERR -

In [7]:
# Processing - 
# Should contain no dups - 
HIERARCHY_DF = HIERARCHY_DF.unique(subset = ['Territory_ID'])

ZTT_TO_TERR = (
   ZIP_DF.join(HIERARCHY_DF,on = 'Territory_ID',how = 'left')
    .with_columns(pl.col("Zip_Code").cast(pl.Utf8).str.zfill(5).alias("ZIP1"))
    .rename({'ZIP1' : 'Zip', 'Territory_ID' : 'Territory'})
    .select(['Zip' ,'Territory' ,'Territory_Name' ,'Region' ,'Region_Name', 'Area', 'Area_Name'])
    .unique(subset = 'Zip')
)

# Export - 
ZTT_TO_TERR.to_pandas().to_parquet(f'{ZIP}zip_to_terr.parquet')